In [7]:
from typing import Annotated
from dotenv import load_dotenv

load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [8]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [9]:
message = {"role": "user", "content": "Who walked on the moon for the first tiem? Print only the name"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first tiem? Print only the name', additional_kwargs={}, response_metadata={}, id='deb71333-5206-45d0-bdee-9e319c8daa14'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--73135fa8-1923-4da3-8ee4-aedb0d79aaa0-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})]

In [10]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot: ", state["messages"][-1].content)

Bot:  Neil Armstrong
Bot:  July 20, 1969
